In [1]:
import interp.cui as cui
import jax.numpy as jnp
import jax

await cui.init(port=6789)  # type: ignore

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.8/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


Loading models from /Users/ckmacleod/rrfs/interpretability_models_jax/
started
Composable UI initialized! Make sure you've run `npm install` and `npm run start` in /interp/app before using


In [3]:
from interp.ui.very_named_tensor import VeryNamedTensor

from interp.model.model_loading import load_model
from interp.tools.log import LogCacheAll

model, params, tok = load_model("jan5_attn_only_two_layers")
text = "[BEGIN] Mr and Mrs Dursley were proud to say that they were perfectly normal. Mr Dursley made drills."

log = LogCacheAll()
token_ids = tok(text, padding=False, return_tensors="jax")["input_ids"]
token_strs = [tok.decode(token_id) for token_id in token_ids[0]]
out, _ = model.apply(params, token_ids, log=log)
# %%

# Let's visualize the attention scores by layer and head
attn = jnp.concatenate(
    [log.get(f"blocks.{i}.attention.attn_probs") for i in range(model.num_layers)]
)  # shape = (layer, head, seq, seq)

vnt_attn = VeryNamedTensor(
    attn,
    dim_names="layer head Q K".split(),
    dim_types="layer head seq seq".split(),
    dim_idx_names=[
        [str(i) for i in range(model.num_layers)],
        [str(i) for i in range(model.num_heads)],
        token_strs,
        token_strs,
    ],
    units="prob",
    title="Attention Probabilities",
)

# Also visualize the model's predictions
logprobs = jax.nn.log_softmax(out[0])
# For demo purposes reduce the amount of data sent - there's better ways to do this
top_token_ids = (-logprobs.max(0)).argsort()[:50]
vnt_preds = VeryNamedTensor(
    logprobs[:, top_token_ids],
    dim_names="seq vocab".split(),
    dim_types="seq vocab".split(),
    dim_idx_names=[token_strs, [tok.decode(token_id) for token_id in top_token_ids]],
    units="logprob",
    title="Model Predictions",
)


In [4]:
# Run this and then click the link that appears.
cui.show_tensors(
    vnt_preds,
    vnt_attn,
)

In [5]:

# %%

handler
init
nameStartup
name untitled
handler
init
nameStartup
name untitled
callback
